<h1>Sulfite Oxidase Deficiency SUOX</h1>
<p>Data from <a href="https://pubmed.ncbi.nlm.nih.gov/36303223/" target="__blank">Li JT, Chen ZX, Chen XJ, Jiang YX. Mutation analysis of SUOX in isolated sulfite oxidase deficiency with ectopia lentis as the presenting feature: insights into genotype-phenotype correlation. Orphanet J Rare Dis. 2022 Oct 27;17(1):392. doi: 10.1186/s13023-022-02544-x. PMID: 36303223; PMCID: PMC9615255.</a></p>

In [1]:
import typing
import os

import hpotk
from phenopackets import Phenopacket

In [2]:
fpath_hpo = 'hpo_data/hp.json'
cache_dir = 'annotations'
fpath_phenopackets = 'phenopackets'

In [3]:
from genophenocorr.preprocessing import configure_caching_patient_creator

In [4]:
hpo: hpotk.ontology.Ontology = hpotk.ontology.load.obographs.load_ontology(fpath_hpo)
    
phenotype_creator = configure_caching_patient_creator(hpo, cache_dir=cache_dir)

In [5]:
from genophenocorr.preprocessing import load_phenopacket_folder

In [6]:
patientCohort = load_phenopacket_folder(fpath_phenopackets, phenotype_creator)

In [7]:
from IPython.display import HTML, display
from genophenocorr.view import CohortViewer

viewer = CohortViewer(hpo)

In [8]:
patientCohort.list_all_variants(10)

[('12_56004589_56004589_C_G', 7),
 ('12_56004039_56004039_G_A', 3),
 ('12_56004485_56004485_C_T', 3),
 ('12_56004765_56004765_G_A', 3),
 ('12_56004905_56004909_ATTGT_A', 2),
 ('12_56004933_56004959_A_ACAATGTGCAGCCAGACACCGTGGCCC', 2),
 ('12_56004771_56004771_A_T', 2),
 ('12_56004273_56004273_G_A', 2),
 ('12_56004192_56004192_G_A', 1),
 ('12_56004161_56004161_A_C', 1)]

In [9]:
patientCohort.list_all_phenotypes()

[('HP:0001250', 28),
 ('HP:0001252', 15),
 ('HP:0001276', 11),
 ('HP:0002071', 11),
 ('HP:0000252', 10),
 ('HP:0012758', 8),
 ('HP:0001083', 7),
 ('HP:0034332', 6)]

In [10]:
patientCohort.list_data_by_tx()

{'NM_001032386.2': Counter({'MISSENSE_VARIANT': 29,
          'FRAMESHIFT_VARIANT': 9,
          'STOP_GAINED': 10}),
 'NM_000456.3': Counter({'MISSENSE_VARIANT': 29,
          'FRAMESHIFT_VARIANT': 9,
          'STOP_GAINED': 10}),
 'NM_001032387.2': Counter({'MISSENSE_VARIANT': 29,
          'FRAMESHIFT_VARIANT': 9,
          'STOP_GAINED': 10})}

In [11]:
len(patientCohort.list_all_patients())

35

In [20]:
from genophenocorr.analysis import CohortAnalysis
from genophenocorr.model import VariantEffect

In [21]:
analysis = CohortAnalysis(patientCohort, 'NM_000456.3', hpo, include_unmeasured=False, 
                          recessive=True, p_val_correction='fdr_bh')

In [22]:
analysis.compare_by_variant_type(VariantEffect.MISSENSE_VARIANT)

Homozygous missense_variant  \
                                                                         Count   
HP:0034332 (Cognitive regression)                                            6   
HP:0001252 (Hypotonia)                                                      10   
HP:0001250 (Seizure)                                                        12   
HP:0002071 (Abnormality of extrapyramidal motor...                           8   
HP:0012758 (Neurodevelopmental delay)                                        4   
HP:0001083 (Ectopia lentis)                                                  3   
HP:0000252 (Microcephaly)                                                    4   
HP:0001276 (Hypertonia)                                                      5   

                                                            \
                                                   Percent   
HP:0034332 (Cognitive regression)                   50.00%   
HP:0001252 (Hypotonia)                              90.91%   
HP:0001250 (Seizure)                                66.67%   
HP:0002071 (Abnormality of extrapyramidal motor...  66.67%   
HP:0012758 (Neurodevelopmental delay)               33.33%   
HP:0001083 (Ectopia lentis)                         27.27%   
HP:0000252 (Microcephaly)                           40.00%   
HP:0001276 (Hypertonia)                             45.45%   

                                                   Heterozygous missense_variant  \
                                                                           Count   
HP:0034332 (Cognitive regression)                                              0   
HP:0001252 (Hypotonia)                                                         2   
HP:0001250 (Seizure)                                                           5   
HP:0002071 (Abnormality of extrapyramidal motor...                             1   
HP:0012758 (Neurodevelopmental delay)                                          0   
HP:0001083 (Ectopia lentis)                                                    3   
HP:0000252 (Microcephaly)                                                      2   
HP:0001276 (Hypertonia)                                                        2   

                                                            \
                                                   Percent   
HP:0034332 (Cognitive regression)                    0.00%   
HP:0001252 (Hypotonia)                              40.00%   
HP:0001250 (Seizure)                                83.33%   
HP:0002071 (Abnormality of extrapyramidal motor...  20.00%   
HP:0012758 (Neurodevelopmental delay)                0.00%   
HP:0001083 (Ectopia lentis)                         75.00%   
HP:0000252 (Microcephaly)                           50.00%   
HP:0001276 (Hypertonia)                             40.00%   

                                                   No missense_variant  \
                                                                 Count   
HP:0034332 (Cognitive regression)                                    0   
HP:0001252 (Hypotonia)                                               3   
HP:0001250 (Seizure)                                                11   
HP:0002071 (Abnormality of extrapyramidal motor...                   2   
HP:0012758 (Neurodevelopmental delay)                                4   
HP:0001083 (Ectopia lentis)                                          1   
HP:0000252 (Microcephaly)                                            4   
HP:0001276 (Hypertonia)                                              4   

                                                                       \
                                                    Percent   p-value   
HP:0034332 (Cognitive regression)                     0.00%  0.023343   
HP:0001252 (Hypotonia)                               42.86%  0.039530   
HP:0001250 (Seizure)                                100.00%  0.082640   
HP:0002071 (Abnormality of extrapyramidal

In [19]:
analysis.compare_by_variant('12_56004589_56004589_C_G')

Homozygous 12_56004589_56004589_C_G  \
                                                                                 Count   
HP:0001252 (Hypotonia)                                                               0   
HP:0001083 (Ectopia lentis)                                                          0   
HP:0012758 (Neurodevelopmental delay)                                                0   
HP:0002071 (Abnormality of extrapyramidal motor...                                   0   
HP:0034332 (Cognitive regression)                                                    0   
HP:0001276 (Hypertonia)                                                              1   
HP:0000252 (Microcephaly)                                                            1   
HP:0001250 (Seizure)                                                                 2   

                                                             \
                                                    Percent   
HP:0001252 (Hypotonia)                                0.00%   
HP:0001083 (Ectopia lentis)                           0.00%   
HP:0012758 (Neurodevelopmental delay)                 0.00%   
HP:0002071 (Abnormality of extrapyramidal motor...    0.00%   
HP:0034332 (Cognitive regression)                     0.00%   
HP:0001276 (Hypertonia)                             100.00%   
HP:0000252 (Microcephaly)                           100.00%   
HP:0001250 (Seizure)                                100.00%   

                                                   Heterozygous 12_56004589_56004589_C_G  \
                                                                                   Count   
HP:0001252 (Hypotonia)                                                                 1   
HP:0001083 (Ectopia lentis)                                                            2   
HP:0012758 (Neurodevelopmental delay)                                                  0   
HP:0002071 (Abnormality of extrapyramidal motor...                                     1   
HP:0034332 (Cognitive regression)                                                      0   
HP:0001276 (Hypertonia)                                                                1   
HP:0000252 (Microcephaly)                                                              1   
HP:0001250 (Seizure)                                                                   4   

                                                             \
                                                    Percent   
HP:0001252 (Hypotonia)                               25.00%   
HP:0001083 (Ectopia lentis)                         100.00%   
HP:0012758 (Neurodevelopmental delay)                 0.00%   
HP:0002071 (Abnormality of extrapyramidal motor...   20.00%   
HP:0034332 (Cognitive regression)                     0.00%   
HP:0001276 (Hypertonia)                              25.00%   
HP:0000252 (Microcephaly)                            33.33%   
HP:0001250 (Seizure)                                 80.00%   

                                                   No 12_56004589_56004589_C_G  \
                                                                         Count   
HP:0001252 (Hypotonia)                                                      14   
HP:0001083 (Ectopia lentis)                                                  5   
HP:0012758 (Neurodevelopmental delay)                                        8   
HP:0002071 (Abnormality of extrapyramidal motor...                          10   
HP:0034332 (Cognitive regression)                                            6   
HP:0001276 (Hypertonia)                                                      9   
HP:0000252 (Microcephaly)                                                    8   
HP:0001250 (Seizure)                                                        22   

                                                                      \
                                                   Percent   p-value   
HP